In [4]:
from datasets import load_dataset
dataset = load_dataset("Cartinoe5930/raw_text_synthetic_dataset_50k", split = "train")

# 역색인

In [16]:
from kiwipiepy import Kiwi
from collections import defaultdict
import math

In [25]:
class InvertedIndex:
    def __init__(self):
        self.reset()

    def reset(self):
        self.index = defaultdict(dict)
        self.kiwi = Kiwi()
        self.document_lengths = {}
        self.total_documents = 0
        self.average_document_length = 0
        self.documents = {}

    def tokenize(self, text):
        return [token.form for token in self.kiwi.tokenize(text)]

    def add_document(self, doc_id, question, answer):
        tokens = self.tokenize(question)
        self.document_lengths[doc_id] = len(tokens)
        self.total_documents += 1
        self.documents[doc_id] = {'question': question, 'answer': answer}

        for token in set(tokens):
            if doc_id not in self.index[token]:
                self.index[token][doc_id] = 0
            self.index[token][doc_id] += tokens.count(token)

        self.average_document_length = sum(self.document_lengths.values()) / self.total_documents

    def calculate_bm25_score(self, query_tokens, doc_id):
        k1 = 1.5
        b = 0.75
        score = 0

        for token in query_tokens:
            if token not in self.index or doc_id not in self.index[token]:
                continue

            tf = self.index[token][doc_id]
            df = len(self.index[token])
            idf = math.log((self.total_documents - df + 0.5) / (df + 0.5) + 1)

            numerator = tf * (k1 + 1)
            denominator = tf + k1 * (1 - b + b * self.document_lengths[doc_id] / self.average_document_length)
            score += idf * numerator / denominator

        return score

    def search(self, query, k=5):
        query_tokens = self.tokenize(query)
        scores = defaultdict(float)

        for token in query_tokens:
            if token in self.index:
                for doc_id in self.index[token]:
                    scores[doc_id] += self.calculate_bm25_score(query_tokens, doc_id)

        top_k = sorted(scores.items(), key=lambda x: x[1], reverse = True)[:k]
        return [(doc_id, score, self.documents[doc_id]) for doc_id, score in top_k]

In [26]:
# add docs
from tqdm import tqdm

index = InvertedIndex()
for idx, data in enumerate(tqdm(dataset)):
    question = data['question']
    answer = data['response']
    index.add_document(idx, question, answer)

100%|██████████| 21900/21900 [00:46<00:00, 471.96it/s]


# 유사 문제 찾기

In [48]:
# 2000
question = dataset[100]['question']
answer = dataset[100]['response']

print(question)
print(answer)

similar_questions = index.search(question, k=15)
for quest in similar_questions:
    print(quest)

2019년에 번역이 진행된 여러 금융 AI 가이드라인 중에서, AI 시스템이 공정하고 투명하며 설명 가능한 이유가 무엇인지, 또한 이러한 시스템을 개발하기 위해 금융 서비스 조직이 어떤 절차를 마련해야 하는지를 설명하는 데 필요한 여러 단계의 추론은 무엇입니까?
AI 시스템이 공정하고 투명하며 설명 가능해야 하는 이유는 다음과 같습니다:

1. **신뢰 구축**: 금융 서비스 분야에서 고객과의 신뢰는 핵심입니다. 공정하고 투명한 AI 시스템은 고객이 시스템의 결정 과정을 이해하고 신뢰할 수 있도록 도와줍니다.

2. **편향 제거**: AI 모델이 훈련 데이터의 편향을 반영하지 않도록 하는 것이 중요합니다. 공정성을 통해 특정 그룹에 대한 시스템의 차별을 방지할 수 있습니다.

3. **규제 준수**: 많은 국가에서는 금융 서비스에 대한 규제가 강화되고 있으며, 이러한 규제를 준수하기 위해서는 AI 시스템의 투명성과 설명 가능성이 필수적입니다.

4. **결정 과정의 이해**: 설명 가능한 AI는 사용자가 시스템의 결정을 더 잘 이해할 수 있도록 하여, 잘못된 결정에 대한 피드백과 개선이 용이합니다.

금융 서비스 조직이 공정하고 투명하며 설명 가능한 AI 시스템을 개발하기 위해 반드시 숙지해야 할 절차의 여러 단계는 다음과 같습니다:

1. **윤리적 프레임워크 정의**: 조직의 AI 시스템이 따라야 할 윤리적 기준을 설정하고, 공정성, 투명성, 설명 가능성에 대한 명확한 가이드라인을 마련합니다.

2. **데이터 관리 및 품질 보증**: 공정성을 위해 다양하고 포괄적인 데이터셋을 수집하고, 데이터의 편향을 평가하며, 필요한 경우 데이터를 정제해야 합니다.

3. **모델 개발 프로세스**: AI 모델 개발 시 공정성과 투명성을 보장할 수 있는 알고리즘과 기법을 적용합니다. 모델의 결정 과정을 설명할 수 있는 방법론을 사용해야 합니다.

4. **테스트 및 검증**: AI 시스템의 공정성, 정확성 및 설명 가능성을 검증하기 위한 테스트 환경을 마련하고, 다양

# 비슷한 문제들을 참고해서 만들게 하자

In [30]:
import openai
from dotenv import load_dotenv
import os

dotenv_path = os.path.join(os.getcwd(), 'KRX', '.env')
load_dotenv(dotenv_path)

openai_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai_key)

In [49]:
system_prompt = """
You are given two pairs of reference questions and reference answers.
Your role is a questioner who make a new question.
When making your questions, consider the following.
1. New Question must require choices such as 'Which is right', 'Which is not right', 'Which is most appropriate', and 'Which is not most appropriate'.
2. You have to make 5 choices, 1 answer choice and 4 wrong choices.
3. The choices must be generated in association with one of several keywords in the reference question and answer.
4. The wrong answer and the right answer are confused, but the wrong answer must be a clear wrong answer.
5. The choices does not deviate from the subject of the problem, but it must be different.
6. The choices sentence must be similar in length.
7. If a person is a financial expert, the person can solve the problem, but if the person is a beginner in financial knowledge, please make the problem with a difficulty that the person cannot solve because it is difficult.
8. Please don't create a problem that can be solved by reading other than the problem.
please write in Korean and you must write the answer on the last line.
"""

user_prompt ="""
### Reference
### Question 1: {}
### Answer 1: {}

### Question 2: {}
### Answer 2: {}

### New Question : 

"""


In [50]:
system_msg = {'role':'system', 'content': system_prompt}
user_msg = {'role':'user', 'content': user_prompt.format(
    similar_questions[0][2]['question'],
    similar_questions[0][2]['answer'],
    similar_questions[1][2]['question'],
    similar_questions[1][2]['answer'])}

response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages = [system_msg, user_msg],
)

result = response.choices[0].message.content
print(result)

2019년에 발표된 AI 시스템의 투명성, 공정성, 설명 가능성에 대한 가이드라인과 관련하여, 금융 서비스 조직이 이러한 가이드라인을 준수하기 위해 반드시 개발해야 할 정책과 절차 중에서 가장 적절한 것은 무엇인가요?

1. 데이터 관리 체계를 구축하고, 주기적인 데이터 감사 및 검증을 실시해야 한다.
2. AI 시스템의 모든 결정 과정을 고객에게 투명하게 공개하지 않는다.
3. 사용자가 AI 시스템의 위험을 이해할 수 있는 교육 프로그램을 제공해야 한다.
4. 편향 검토 없이 단일 데이터셋만을 사용하여 AI 모델을 개발한다.
5. 사용자의 피드백을 반영할 수 있는 메커니즘을 마련해야 한다.

정답: 1. 데이터 관리 체계를 구축하고, 주기적인 데이터 감사 및 검증을 실시해야 한다.


# 10개정도 만들어서 정성평가 해보자

In [51]:
# 8.25 -> 
import random
from tqdm import tqdm

random_choice = random.sample(range(len(dataset)), 10)
print(random_choice)

new_questions = []
for i in tqdm(sorted(random_choice)):
    question = dataset[i]['question']
    answer = dataset[i]['response']
    system_msg = {'role':'system', 'content': system_prompt}

    similar_questions = index.search(question, k=2)

    user_msg = {'role':'user', 'content': user_prompt.format(
        similar_questions[0][2]['question'],
        similar_questions[0][2]['answer'],
        similar_questions[1][2]['question'],
        similar_questions[1][2]['answer'])}

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages = [system_msg, user_msg],
    )

    result = response.choices[0].message.content
    new_questions.append(result)
    
    print(result)
    print("*"*100)


[10150, 2895, 2059, 18466, 13909, 2330, 16428, 1380, 20692, 6976]


 10%|█         | 1/10 [00:17<02:39, 17.71s/it]

APDA와 ASIDS 두 조직의 중앙 집중식 은행 시스템이 재정 자원 관리에 미치는 영향을 비교할 때, 어느 접근 방식이 가장 적절한지에 대해 선택하십시오. 다음 중 가장 적합한 설명은 무엇입니까?

1. APDA는 경제적 성장에 주로 초점을 맞추며, ASIDS는 전적으로 위기 관리에만 의존합니다.
2. ASIDS는 투자 포트폴리오를 경제 문제에 집중하여 효율적으로 관리합니다.
3. APDA는 지속 가능한 발전을 위해 자원 관리에서 장기적인 접근 방식을 취합니다.
4. 두 조직 모두 비슷한 방식으로 재정 자원을 관리하므로 차별화된 전략이 필요하지 않습니다.
5. ASIDS는 주로 고위험 프로젝트에만 집중하며 재정 관리를 소홀히 합니다.

정답: 3. APDA는 지속 가능한 발전을 위해 자원 관리에서 장기적인 접근 방식을 취합니다.
****************************************************************************************************


 20%|██        | 2/10 [00:34<02:16, 17.10s/it]

런던의 금세공인들과 피렌체의 메디치 가문이 은행 시스템 발전에 기여한 바가 큽니다. 이 두 혁신이 아시아와 유럽의 상업은행에 미친 영향 중 어떤 것이 가장 적절한가요? 

1. 아시아의 은행들은 메디치 가문의 방식을 효과적으로 복제하였다.
2. 유럽의 은행들은 금세공인의 시스템을 완전히 무시하였다.
3. 아시아의 은행들은 초기 유럽 모델을 도입하여 그 발전에 기여하였다.
4. 피렌체의 메디치 가문은 아닌 다른 가문과 연합하여 상업은행 모델을 제안하였다.
5. 런던의 금세공인은 금융 안정성과 신뢰도를 떨어뜨리는 방법들을 도입하였다. 

정답: 3. 아시아의 은행들은 초기 유럽 모델을 도입하여 그 발전에 기여하였다.
****************************************************************************************************


...

말레이시아 이슬람 은행의 자본 적정성 요건에 대해 설명할 때, 아래의 선택지 중에서 가장 적합한 것을 고르시오. 

1. 말레이시아 이슬람 은행은 최소 5000만 링깃의 자본금이 필요하다.
2. 최소 1500만 링깃이 실현된 자본으로 지급되어야 한다.
3. 자본 적정성 요건은 다른 국가와 동일한 기준을 따른다.
4. 은행 안정성을 보장하기 위해 자본 적정성이 필요하다.
5. 말레이시아는 이자 기반 금융 시스템만을 사용한다.

정답: 2
****************************************************************************************************


 90%|█████████ | 9/10 [02:34<00:16, 16.33s/it]

수출 신용 기관이 직면하는 다양한 위험 요인들이 이들이 제공하는 금융 서비스의 질에 미치는 영향을 고려할 때, 어느 정도의 위험 관리가 필요한지에 대해 답해 보세요. 다음 중 가장 올바른 설명은 무엇인가요?

1. 수출 신용 기관은 경제 불확실성이 높을 때 위험 관리를 강화해야 한다.
2. 정치적 불안정성과 환율 변동은 수출 신용 기관의 서비스에 영향이 없으므로 걱정할 필요가 없다.
3. 위험 관리 절차가 없으면 서비스의 수익성에 부정적인 영향을 미칠 수 있다.
4. 글로벌 공급망 변동성은 수출 신용 기관의 서비스에는 전혀 영향을 주지 않는 문제이다.
5. 기후 변화와 자연재해는 수출 신용 기관의 서비스 개선에 집중해야 할 주제가 아니다.

정답: 1. 수출 신용 기관은 경제 불확실성이 높을 때 위험 관리를 강화해야 한다.
****************************************************************************************************


100%|██████████| 10/10 [02:52<00:00, 17.23s/it]

프랑스와 스페인에서의 암호화폐 채굴 작업이 에너지 소비에 미치는 영향에 대해 판단할 때, 다음 중 어떤 진술이 올바른가요?

1. 프랑스의 암호화폐 채굴은 원자력 에너지로 인해 상대적으로 환경적으로 지속 가능한 운영이 이루어진다.
2. 스페인의 채굴 작업은 주로 석탄과 같은 화석 연료에 의존하여 진행된다.
3. 프랑스는 재생 가능 에너지원의 부족으로 암호화폐 채굴 작업에서 어려움을 겪고 있다.
4. 스페인은 태양광 및 풍력 발전의 활용도가 높아 채굴 작업의 탄소 배출이 적다.
5. 프랑스의 암호화폐 채굴은 전력 소비 증가로 인해 지역사회에 긍정적인 영향을 미치고 있다.

정답: 1
****************************************************************************************************
